In [1]:
import torch
import os
from typing import List
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import albumentations as A
import numpy as np
import albumentations.pytorch as al_pytorch
from typing import Dict, Tuple
import pytorch_lightning as pl
torch.__version__

'1.11.0+cpu'

## Number of images

In [2]:
# DATA_ROOT_PATH:str = "/kaggle/input/anime-sketch-colorization-pair/data/"
# NO_OF_IMAGES:int = 5 # Limit the number of images
# #
# train_path:str = f"{DATA_ROOT_PATH}train/"
# train_images:List[str] = [ f"{train_path}{x}" for x in os.listdir(train_path) ][:NO_OF_IMAGES]
# test_path:str = f"{DATA_ROOT_PATH}val/"
# test_images:List[str] = [ f"{test_path}{x}" for x in os.listdir(test_path) ][:NO_OF_IMAGES]
# #
# print(" Nb of train images : " + str(len(train_images)))
# print(" Nb of test images : " + str(len(test_images)))

## Let's plot a few images

In [3]:
# plt.figure(figsize=(15,8))
# for i, img_path in enumerate(train_images[:9]):
#     im = Image.open(img_path)
#     plt.subplot(3,3,i+1)
#     plt.imshow(im)

# Create Model

In [4]:
!git clone -b 17-new_unet_architecture https://github.com/nmud19/thesisGAN.git

Cloning into 'thesisGAN'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 253 (delta 6), reused 5 (delta 5), pack-reused 241
Receiving objects: 100% (253/253), 13.84 MiB | 29.04 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [5]:
os.chdir("thesisGAN")
from app.discriminator import patch_gan
from app.generator import unetGen
from app.consume_data import consume_data
from app.model import lit_model

# Set up gen and discriminator

In [6]:
generator = unetGen.Generator(in_channels=3)
discriminator = patch_gan.Discriminator(in_channels=3)


# Setup the data

In [7]:
anime_sketch_data_module = consume_data.AnimeSketchDataModule(
    data_dir="/kaggle/input/anime-sketch-colorization-pair/data/",
    train_num_images=8,
    val_num_images=6,
    train_batch_size=8,
    val_batch_size=6,
)

# Add the pytorch lightning script

In [8]:
# Trainer
from app.model.lit_model import Pix2PixLitModule

model = Pix2PixLitModule(
    generator=generator,
    discriminator=discriminator,
    use_gpu=False
)
# Trainer
# epoch_inference_callback = lit_model.EpochInference(valid_dataloader, use_gpu=False)
# checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint()
logger = pl.loggers.TensorBoardLogger("tb_logs", name="pix2pix_lightning_model")
trainer = pl.Trainer(
    #fast_dev_run=True,
    max_epochs=4,
    logger=logger,
#     callbacks=[
#         epoch_inference_callback,
#         checkpoint_callback,
#         pl.callbacks.TQDMProgressBar(refresh_rate=10)
#     ],
    default_root_dir="chk",
    #tpu_cores=8,
    #accelerator='gpu', 
    #devices=1,
    #gpus=-1
)
trainer.fit(
    model=model,
    datamodule=anime_sketch_data_module
)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'generator' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['generator'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'discriminator' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['discriminator'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."


The train test dataset lengths are :  8 6


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1937: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [9]:
!ls 

01-data-exploration.ipynb   docs	      sample_data
README.md		    helpers	      tb_logs
app			    notebooks	      tests
conditional-gan-test.ipynb  requirements.txt


In [10]:
%load_ext tensorboard
%tensorboard --logdir tb_logs